In [46]:
from pathlib import Path
import re
import requests
import time
import pandas as pd
from requests_html import HTML
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

In [47]:
BASE_DIR=Path.cwd()
DATA_DIR=BASE_DIR/ "data" #os.path.join(BASE_DIR,'data')
if not DATA_DIR.exists():
    DATA_DIR.mkdir(exist_ok=True)
product_category_links_output=DATA_DIR /"category-product.csv"    
product_output=DATA_DIR /"product.csv"   

In [48]:
options=Options()
options.add_argument("--headless")

driver =webdriver.Chrome(options=options)

In [49]:
categories=[
    {"name":"books","url":"https://www.amazon.in/gp/bestsellers/books/"},
    {"name":"gift-cards","url":"https://www.amazon.in/gp/bestsellers/gift-cards/"}
]

In [58]:
regex_options=[
    r"https://www.amazon.in/gp/product/(<product_id>[\w-]+)/",
    r"https://www.amazon.in/dp/(<product_id>[\w-]+)/"

]

def extract_product_id_from_url(url):
    product_id=None
    for regex_str in regex_options:
        regex=re.compile(regex_str)
        match=regex.match(url)
        if match !=None:
            try:
                product_id=match['product_id']
            except:
                pass
    return product_id       

In [59]:
def clean_page_links(page_links=[]):
    final_page_links=[]
    for url in page_links:
        product_id=extract_product_id_from_url(url)
        if product_id !=None:
            final_page_links.apend({"url":url,"product_id":product_id})
    return final_page_links


In [60]:
def scrace_category_product_links(categories=[]):
    all_product_links=[]
    for category in categories:
        time.sleep(1.5)
        url=category.get("url")
        driver.get(url)
        body_el=driver.find_element_by_css_selector("body")
        html_str=body_el.get_attribute("innerHTML")
        html_obj=HTML(html=html_str)
        page_links=[f"https://www.amazon.in{x}" for x in html_obj.links if x.startswith("/")]
        cleaned_links=clean_page_links(page_links=page_links)
        all_product_links+=cleaned_links
    return all_product_links
         


In [68]:
def extract_categories_and_save(categories=categories):
    all_product_links=scrace_category_product_links(categories)
    category_df=pd.DataFrame(all_product_links)
    category_df.to_csv(product_category_links_output,index=False)

In [69]:
extract_categories_and_save(categories=categories)

In [70]:
def scrape_product_page(url,title_lookup="#productTitle",price_lookup="#priceblock_ourprice"):

    driver.get(url)
    time.sleep(0.5)
    body_el=driver.find_element_by_css_selector("body")
    html_str=body_el.get_attribute("innerHTML")
    print(html_str)
    html_obj=HTML(html=html_str)
    product_title=html_obj.find(title_lookup,first=True)
    product_price=html_obj.find(price_lookup,first=True)
    return product_title,product_price


In [71]:
def perform_scrape(cleaned_items=[]):
    data_extracted=[]
    for obj in cleaned_items:
        link=obj['url']
        product_id=obj['product_id']
        title,price=(None,None)
        try:
            title,price=scrape_product_page(link)
        except:
            pass
        if title!=None and price !=None:
            print(link,title,price)    
        product_data={
            "url":link,
            "product_id":product_id,
            "title":title,
            "price":price
        }   
        data_extracted.append(product_data) 


In [72]:
#extracted_data=perform_scrape(cleaned_items=cleaned_links)

In [73]:
#print(extracted_data)

In [ ]:
def row_scrape_event(cleaned_items=[]):
    data_extracted=[]
    for obj in cleaned_items:
        link=obj['url']
        product_id=obj['product_id']
        title,price=(None,None)
        try:
            title,price=scrape_product_page(link)
        except:
            pass
        if title!=None and price !=None:
            print(link,title,price)    
        product_data={
            "url":link,
            "product_id":product_id,
            "title":title,
            "price":price
        }   
        data_extracted.append(product_data) 

In [74]:
df=pd.read_csv(product_category_links_output)
df.head()

EmptyDataError: No columns to parse from file